In [1]:
import sys
sys.path.append("/work/b0990106x/trl/vc")
import importlib
import vc
importlib.reload(vc)
import torch
from vc.trainer_encodec_vc_inference import get_ar_prediction_v3
from types import SimpleNamespace
from transformers import BartForConditionalGeneration, AutoModelForCausalLM, AutoTokenizer
from NISQA.nisqa.NISQA_model import nisqaModel
from datasets import load_from_disk
from trl import DPOTrainer, DPOConfig, AutoModelForSeq2SeqLMWithValueHead, create_reference_model
from vc.encodec_model.nar_bart_model import NARBartForConditionalGeneration
from datetime import datetime
import os
import numpy as np

/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define paths and device
base_path = "/work/b0990106x/trl"
device = "cuda" if torch.cuda.is_available() else "cpu"

now = datetime.now()
ts = now.strftime("%m%d-%H%M")
print("timestamp:", ts)

model_output_dir = f"{base_path}/model_output/{ts}"
agent_input_dir = f"{base_path}/data-encodec"
agent_output_dir = f"{base_path}/output/{ts}"
env_input_dir = agent_output_dir
env_output_dir = agent_input_dir

if not os.path.exists(model_output_dir):
    os.makedirs(model_output_dir)

timestamp: 0620-1700


In [3]:
ar_checkpoint = "lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans"
nar_checkpoint = "lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans"

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(ar_checkpoint, return_dict=True)
model_ref = create_reference_model(model)
nar_model = NARBartForConditionalGeneration.from_pretrained(nar_checkpoint)
ar_tokenizer = AutoTokenizer.from_pretrained(ar_checkpoint)
nar_tokenizer = AutoTokenizer.from_pretrained(nar_checkpoint)
ar_tokenizer.pad_token = ar_tokenizer.eos_token

/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
import json
from datasets import Dataset

# load the dpo_data.json file
with open("dpo_data_all.json") as f:
    dpo_data = json.load(f)

# load the dataset
dataset = Dataset.from_dict(dpo_data)
print(dataset)

## TODO: Split the dataset into training and validation sets
dataset_dict = dataset.train_test_split(test_size=0.1)

# load the training and validation datasets
train_dataset = dataset_dict["train"]
val_dataset = dataset_dict["test"]
print("train_dataset", train_dataset)
print("val_dataset", val_dataset)

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 9254
})
train_dataset Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 8328
})
val_dataset Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 926
})


In [5]:
import os
import wandb
wandb.login()

os.environ["WANDB_NOTEBOOK_NAME"] = "trl"
wandb.init(project="trl")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: b09901066 (b09901066_alan). Use `wandb login --relogin` to force relogin


In [6]:
training_args = DPOConfig(
    beta=0.02,
    output_dir=f"{model_output_dir}",
    generate_during_eval = True,
)

trainer = DPOTrainer(
    model=model,
    ref_model=model_ref,
    args=training_args,
    tokenizer=ar_tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# train
trainer.train()

# save the model
trainer.save_model(f"{model_output_dir}/dpo_model")

model.config.to_json_file(f"{model_output_dir}/dpo_model/config.json")
ar_tokenizer.save_pretrained(f"{model_output_dir}/dpo_model")

/work/b0990106x/trl/trl/trainer/dpo_trainer.py:362: UserWarning: `max_length` is not set in the DPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/work/b0990106x/trl/trl/trainer/dpo_trainer.py:375: UserWarning: `max_prompt_length` is not set in the DPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/work/b0990106x/trl/trl/trainer/dpo_trainer.py:388: UserWarning: When using an encoder decoder architecture, you should set `max_target_length` in the DPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/work/b0990106x/trl/trl/trainer/dpo_trainer.py:410: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
Map: 100%|██████████| 926/926 [00:01<00:0

Step,Training Loss
500,0.166100
1000,0.092800
1500,0.066500
2000,0.033600
2500,0.026600
3000,0.010400


('/work/b0990106x/trl/model_output/0620-1700/dpo_model/tokenizer_config.json',
 '/work/b0990106x/trl/model_output/0620-1700/dpo_model/special_tokens_map.json',
 '/work/b0990106x/trl/model_output/0620-1700/dpo_model/vocab.json',
 '/work/b0990106x/trl/model_output/0620-1700/dpo_model/merges.txt',
 '/work/b0990106x/trl/model_output/0620-1700/dpo_model/added_tokens.json',
 '/work/b0990106x/trl/model_output/0620-1700/dpo_model/tokenizer.json')

In [2]:
import sys
sys.path.append("/work/b0990106x/trl/vc")
import importlib
import vc
importlib.reload(vc)
import torch
from vc.trainer_encodec_vc_inference import get_ar_prediction_v3
from types import SimpleNamespace
from transformers import BartForConditionalGeneration, AutoModelForCausalLM, AutoTokenizer
from NISQA.nisqa.NISQA_model import nisqaModel
from datasets import load_from_disk
from trl import DPOTrainer, DPOConfig, AutoModelForSeq2SeqLMWithValueHead, create_reference_model
from vc.encodec_model.nar_bart_model import NARBartForConditionalGeneration
from datetime import datetime
import os

In [3]:
# Define paths and device
base_path = "/work/b0990106x/trl"
device = "cuda" if torch.cuda.is_available() else "cpu"

# now = datetime.now()
# ts = now.strftime("%m%d-%H%M")
# print("timestamp:", ts)
ts = "beta2_v2"

agent_output_dir = f"{base_path}/output/{ts}"

if not os.path.exists(agent_output_dir):
    os.makedirs(agent_output_dir)
    

In [4]:
# Prepare Model

all_src_encodec_layers = []
all_src_encodec = []
all_instruction = []

layer_len = 8
# data_len = len(dataset)


args_predict = SimpleNamespace(output_path=f"{base_path}/output/{ts}/example.wav", seed=0, device=device)
agent_input_dir = f"{base_path}/data-encodec"
test_dataset = load_from_disk(agent_input_dir)
data_len = len(test_dataset)
print("data_len:", data_len)

for i in range(layer_len):
    all_src_encodec_layers.append(test_dataset[f"src_encodec_{i}"])

for i in range(data_len):
    src_encodec = []
    for j in range(layer_len):
        src_encodec.append(all_src_encodec_layers[j][i])
    all_src_encodec.append(src_encodec)

    all_instruction.append(test_dataset["instruction"][i])

data_len: 9957


In [5]:
ar_checkpoint = "lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans"
nar_checkpoint = "lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans"

nar_model = NARBartForConditionalGeneration.from_pretrained(nar_checkpoint)
ar_tokenizer = AutoTokenizer.from_pretrained(ar_checkpoint)
nar_tokenizer = AutoTokenizer.from_pretrained(nar_checkpoint)
ar_tokenizer.pad_token = ar_tokenizer.eos_token

/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
# Function to calculate reward
import time
def get_reward(output_path):
    args_nisqa = {
        "mode": "predict_file",
        "pretrained_model": f"{base_path}/NISQA/weights/nisqa.tar",
        "deg": output_path,
        "data_dir": None,
        "output_dir": f"{base_path}/NISQA/result/",
        "csv_file": None,
        "csv_deg": None,
        "num_workers": 0,
        "bs": 1,
        "ms_channel": None,
    }
    args_nisqa["tr_bs_val"] = args_nisqa["bs"]
    args_nisqa["tr_num_workers"] = args_nisqa["num_workers"]
    nisqa = nisqaModel(args_nisqa)
    try:
        prediction = nisqa.predict()
        reward = float(prediction["mos_pred"].iloc[0])
        print("Reward:", reward)
        return reward
    except Exception as e:
        print("Error:", e)
        print("get_reward function end ___________________________")
        return None
    
def process_and_get_scores(model, nar_model, ar_tokenizer, nar_tokenizer, src_encodec, instruction, episode_counter=0):
    temp, decode_ar,output_path_ckpt = get_ar_prediction_v3(args_predict, model, nar_model, ar_tokenizer, nar_tokenizer, src_encodec, instruction, episode_counter)
    list_decode_ar = decode_ar.flatten().tolist()
    time.sleep(0.5)
    reward = get_reward(output_path_ckpt)
    return reward

In [7]:
# Load trained model
trained_model = BartForConditionalGeneration.from_pretrained(f"{base_path}/model_output/beta2/dpo_model")
model = BartForConditionalGeneration.from_pretrained(ar_checkpoint, return_dict=True)


Some weights of the model checkpoint at /work/b0990106x/trl/model_output/beta2/dpo_model were not used when initializing BartForConditionalGeneration: ['v_head.summary.weight', 'v_head.summary.bias']
- This IS expected if you are initializing BartForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
# Initialize lists to store rewards
old_model_rewards = []
trained_model_rewards = []

In [9]:
i = 0 
count_rewards = 0
target_rewards = 1000

while count_rewards < target_rewards:
    if i >= data_len:
        print("Exceeded initial data length.")
        break
    instruction = all_instruction[i]
    src_encodec = all_src_encodec[i]
    size_of_packed_input = (len(src_encodec[0]) + len(ar_tokenizer(instruction)["input_ids"][1:-1]) + 3)
    
    if size_of_packed_input <= 1024 and size_of_packed_input > 4:
        # Process with old model
        model.to(device)
        old_model_reward = process_and_get_scores(model, nar_model, ar_tokenizer, nar_tokenizer, src_encodec, instruction, episode_counter=i)
        old_model_rewards.append(old_model_reward)

        # Process with trained model
        trained_model.to(device)
        trained_model_reward = process_and_get_scores(trained_model, nar_model, ar_tokenizer, nar_tokenizer, src_encodec, instruction, episode_counter=i)
        trained_model_rewards.append(trained_model_reward)

        count_rewards += 1
    else:
        print(f"Skipping data point {i} due to insufficient packed input size.")
    i += 1



/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Episode 0 : audio saved to  /work/b0990106x/trl/output/beta2_v2/example_save_0.wav


/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


Reward: 2.3696582317352295


/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


...

/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Episode 11 : audio saved to  /work/b0990106x/trl/output/beta2_v2/example_save_11.wav


/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


Error: n_wins 1348 > max_length 1300 --- /work/b0990106x/trl/output/beta2_v2/example_save_11.wav. Increase max window length ms_max_segments!
get_reward function end ___________________________
Episode 12 : audio saved to  /work/b0990106x/trl/output/beta2_v2/example_save_12.wav
Reward: 1.8696385622024536


KeyboardInterrupt: 

In [ ]:
print("Old Model Rewards:", old_model_rewards)
print("Trained Model Rewards:", trained_model_rewards)

Old Model Rewards: [None, 1.8717782497406006, None, 2.757830858230591, 1.6322604417800903, None, None, 2.7113113403320312, None, 2.6343016624450684, 2.1417410373687744, None, 1.6103763580322266, 1.5146719217300415, None, None, 2.137955665588379, 2.2793071269989014, 1.9043837785720825, 2.251277446746826, None, 2.208383321762085, 2.401906967163086, 2.4635074138641357, 2.303755044937134, 2.552902936935425, 2.5948641300201416, None, 2.0390915870666504, 2.6507956981658936, None, None, 2.429182291030884, None, 1.006941318511963, 3.001690626144409, 2.018461227416992, 2.4022371768951416, None, 2.87459659576416, 1.7149690389633179, None, 2.4751648902893066, 2.767028570175171, None, 2.1119844913482666, None, None, None, None, 1.8746711015701294, None, 2.250643014907837, 2.07224702835083, 2.6481125354766846, 1.292348027229309, 2.2126989364624023, None, 1.932268738746643, 2.022994041442871, None, None, None, None, 2.479299306869507, 2.460249423980713, 2.8040478229522705, None, 1.8452807664871216, 

In [ ]:
import numpy as np
import json
# Filter out None values
filtered_old_model_rewards = [r for r in old_model_rewards if r is not None]
filtered_trained_model_rewards = [r for r in trained_model_rewards if r is not None]

# Calculate and print metrics
def calculate_metrics(rewards):
    metrics = {
        "mean": np.mean(rewards),
        "median": np.median(rewards),
        "std_dev": np.std(rewards),
        "variance": np.var(rewards),
        "min": np.min(rewards),
        "max": np.max(rewards),
        "25th_percentile": np.percentile(rewards, 25),
        "75th_percentile": np.percentile(rewards, 75),
    }
    return metrics

old_model_metrics = calculate_metrics(filtered_old_model_rewards)
trained_model_metrics = calculate_metrics(filtered_trained_model_rewards)

# print("Old Model Rewards:", old_model_rewards)
print("Old Model Metrics:", old_model_metrics)
# print("Trained Model Rewards:", trained_model_rewards)
print("Trained Model Metrics:", trained_model_metrics)
metrics = {
    "old_model": old_model_metrics,
    "trained_model": trained_model_metrics,
    "old_model_rewards": old_model_rewards,
    "trained_model_rewards": trained_model_rewards,
}

with open(f"{base_path}/output/{ts}/metrics.json", "w") as f:
    json.dump(metrics, f, indent=4)


Old Model Metrics: {'mean': 2.3333320349555677, 'median': 2.3188223838806152, 'std_dev': 0.5335566019872158, 'variance': 0.28468264752414413, 'min': 0.7593159079551697, 'max': 4.652859687805176, '25th_percentile': 1.9992775917053223, '75th_percentile': 2.650754451751709}
Trained Model Metrics: {'mean': 1.611536655905684, 'median': 1.584710955619812, 'std_dev': 0.3741507654929876, 'variance': 0.1399887953189886, 'min': 0.6328645944595337, 'max': 3.344425916671753, '25th_percentile': 1.3577812910079956, '75th_percentile': 1.818842202425003}


In [10]:
temp_reward = get_reward("/work/b0990106x/trl/output/beta05/example_save_1.wav")
print("temp_reward:", temp_reward)

Error: n_wins 1348 > max_length 1300 --- /work/b0990106x/trl/output/beta05/example_save_1.wav. Increase max window length ms_max_segments!
get_reward function end ___________________________
temp_reward: None


/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)
